#### Predator-Prey Mathematical Spec:
<br>

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} - \epsilon * population_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad \textrm{'reproduction_rate'} \\
\epsilon: \quad \textrm{'death_rate'} \\
\beta: \quad \textrm{'consumption_rate'} \\
\gamma: \quad \textrm{'growth_rate'} \\
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha), and dies at a rate proportional to the population size $\epsilon$ (epsilon).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

# Dependencies

In [16]:
# Standard libraries: https://docs.python.org/3/library/
import math
import pandas as pd
import plotly.express as px 

In [2]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [17]:
pd.options.plotting.backend = "plotly"

# State Variables & System Parameters

In [4]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.01],
    'death_rate': [0.01],
    'consumption_rate': [0.01],
    'growth_rate': [10.0],
}

# Policy Functions

In [5]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_growth(params, substep, state_history, previous_state):
    delta_food = params['growth_rate']
    return {'delta_food': delta_food}

# State Update Functions

In [6]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)

# Partial State Update Blocks

In [7]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

# Configuration

In [8]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [10]:
sim_config = config_sim({
    'N': 1,
    'T': range(1000),
    'M': system_params
})

experiment = Experiment()

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [11]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1000, 4, 1, 2)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.03s


In [13]:
simulation_result = pd.DataFrame(raw_result)

In [14]:
df = simulation_result.copy()
# df = df[df.simulation == 1]
df

,population,food,simulation,subset,run,substep,timestep
0,50,1000.00,0,0,1,0,0
1,60,1009.50,0,0,1,1,1
2,70,1018.90,0,0,1,1,2
3,80,1028.20,0,0,1,1,3
4,90,1037.40,0,0,1,1,4
...,...,...,...,...,...,...,...
996,995,994.99,0,0,1,1,996
997,995,995.04,0,0,1,1,997
998,996,995.09,0,0,1,1,998
999,996,995.13,0,0,1,1,999


In [18]:
df.plot(kind='line', x='timestep', y=['population','food'])